In [1]:
data_var = '2023-12-17'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9718,2023-12-17,Japão B.League,03:05,Ibaraki Robots,Gunma,2.53,1.48,155.5,1.85,1.85,3.5,2.00,1.72,MafW5zdN,0.395257,0.675676,0.540541,0.540541,0.070933,271.520,25.883829,0.095329,0.0,0.000000,NaN,245.48,3.766,0.160717,0.042676,-57.0,214.108,225.871350,1.054941,1.8,1.643168,0.912871,101.91,2.532,2.384496,0.941744,10.0,68,79,3.61,1.29,284.352,352.300,0.370305,0.000000,0.106446,-0.70,-0.140,-10.928571,0.0,0.0,0.0,1.79,0.358,1.340782,0.0,0.0,0.0
9719,2023-12-17,Japão B2.League,02:00,Kumamoto,Koshigaya Alphas,2.24,1.60,158.5,1.85,1.85,1.5,2.09,1.66,0hSEzUer,0.446429,0.625000,0.540541,0.540541,0.071429,114.624,26.785972,0.233686,3.0,0.000000,0.000000,86.40,1.384,0.278352,0.201122,30.0,115.216,40.472071,0.351271,1.8,1.643168,0.912871,80.30,1.556,0.532334,0.342117,42.0,80,73,1.08,1.10,0.000,0.000,0.235702,0.000000,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9720,2023-12-17,Japão B2.League,01:30,Fukushima,Nara,1.57,2.31,155.5,1.85,1.85,-4.5,2.01,1.72,rDumuAmR,0.636943,0.432900,0.540541,0.540541,0.069843,239.082,25.663597,0.107342,1.2,1.643168,1.369306,257.84,2.858,0.254500,0.089048,2.0,244.942,85.929788,0.350817,1.2,1.643168,1.369306,291.51,3.400,1.139320,0.335094,-31.0,88,79,2.93,3.69,0.000,226.970,0.269721,0.000000,0.109952,0.00,0.000,inf,0.0,0.0,0.0,0.92,0.184,7.119565,0.0,0.0,0.0
9721,2023-12-17,Japão B2.League,01:00,Iwate,Yamagata Wyverns,1.95,1.77,151.5,1.85,1.85,1.5,1.82,1.88,4MtqtjYK,0.512821,0.564972,0.540541,0.540541,0.077792,210.552,137.985554,0.655351,1.2,1.643168,1.369306,403.20,3.290,2.562635,0.778917,-2.0,403.142,166.992221,0.414227,0.0,0.000000,NaN,441.96,4.904,1.236196,0.252079,-74.0,64,87,6.30,5.08,0.000,0.000,0.068430,0.000000,0.022933,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9722,2023-12-17,Japão B.League,03:05,SeaHorses Mikawa,Toyama,1.12,5.56,166.5,1.87,1.83,-13.5,2.03,1.70,WYq6aYYj,0.892857,0.179856,0.534759,0.546448,0.072713,172.150,70.997271,0.412415,2.4,1.341641,0.559017,135.29,2.018,0.742071,0.367726,50.0,427.718,190.870833,0.446254,0.6,1.341641,2.236068,703.50,5.780,3.426675,0.592850,-56.0,83,67,1.63,10.50,158.498,416.786,0.939986,0.015289,0.125118,0.60,0.120,1.000000,0.0,0.0,0.0,-1.63,-0.326,-13.987730,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917,2023-12-17,Eua Ncaa,16:00,Fairfield,Wagner,1.36,3.33,130.5,1.80,1.95,-7.5,2.10,1.67,jZKU4VrN,0.735294,0.300300,0.555556,0.512821,0.035594,263.216,200.339921,0.761124,1.8,1.643168,0.912871,156.11,3.490,2.652706,0.760088,7.0,299.626,416.430061,1.389833,1.8,1.643168,0.912871,94.99,5.506,8.140506,1.478479,-31.0,67,59,2.33,1.61,0.000,441.246,0.594030,0.056569,0.161303,0.00,0.000,inf,0.0,0.0,0.0,-0.36,-0.072,-32.361111,0.0,0.0,0.0
9918,2023-12-17,Eua Ncaa,16:00,Longwood,VMI,1.04,13.50,141.5,1.91,1.91,-18.5,2.05,1.71,G00WaDRi,0.961538,0.074074,0.523560,0.523560,0.035613,98.760,22.738372,0.230239,3.0,0.000000,0.000000,131.20,1.290,0.219886,0.170455,80.0,260.282,173.513264,0.666636,0.0,0.000000,NaN,520.80,4.566,3.246226,0.710956,-59.0,80,56,1.64,9.30,0.000,291.128,1.211905,0.000000,0.127881,0.00,0.000,inf,0.0,0.0,0.

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V1 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9887,07:15,Filipinas Copa Commissioners,TNT Tropang Giga,San Miguel Beermen,2.01,Back Home


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
9719,02:00,Japão B2.League,Kumamoto,Koshigaya Alphas,2.24,Back Home
9721,01:00,Japão B2.League,Iwate,Yamagata Wyverns,1.95,Back Home
9726,06:05,Japão B.League,Ryukyu,Hiroshima D.,1.66,Back Home
9736,11:30,Polônia Liga De Basquete,Torun,Slask Wroclaw,1.68,Back Home
9745,08:00,Grécia A1 Feminina,Proteas Voulas F,Iraklis F,2.12,Back Home
9748,13:00,Hungria Nacional Feminino,UNI Gyor F,Sopron F,1.68,Back Home
9751,15:30,Israel Superliga,Ironi Kiryat Ata,Hapoel Beer Sheva,2.08,Back Home
9754,14:00,Itália Série A2,Agrigento,Monferrato,1.66,Back Home
9761,14:00,Itália Série A1 Feminina,Faenza F,Sassari F,2.31,Back Home
9762,14:00,Itália Série A1 Feminina,San Martino F,Campobasso F,1.63,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
9770,07:00,Turquia Superliga,Manisa,Bursaspor,1.89,Back Home
9782,23:10,Mundo Liga Dos Campeões Das Américas,Halcones de Xalapa,Gladiadores,1.84,Back Home
9892,12:00,Finlândia Divisão I Feminina,LePy F,Turun NMKY F,1.88,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9726,06:05,Japão B.League,Ryukyu,Hiroshima D.,1.66,Back Home
9736,11:30,Polônia Liga De Basquete,Torun,Slask Wroclaw,1.68,Back Home
9835,14:00,Catar Liga De Basquete Do Catar,Catar,Al Ahli,2.68,Back Home
9836,08:35,China Cba,Beijing,Xinjiang,1.87,Back Home
9872,18:00,Eua Ncaa,Abilene Christian,UTEP,1.80,Back Home
9874,18:00,Eua Ncaa,Eastern Kentucky,Northern Kentucky,1.73,Back Home
9887,07:15,Filipinas Copa Commissioners,TNT Tropang Giga,San Miguel Beermen,2.01,Back Home
9897,16:00,Eua Ncaa,Nebraska O.,Stetson,1.91,Back Home
9912,01:59,Eua Ncaa,Howard,Jackson State,1.63,Back Home
